<h1>Numpy and Pandas Home Assignment</h1>
In this assignment, we will use pandas and numpy to do some fairly basic analysis on equities.

<h3>Imports</h3>

In [1]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import pandas_datareader.data as web
# import yfinance as yf

<h3>Get data on the following four stocks from yahoo finance</h3>
<li>MS, GS, JPM, C
<li>Use the date range 2013/1/1/ to 2018/9/24

In [2]:
start = datetime.datetime(2013,1,1)
end = datetime.datetime(2018,9,24)
ms = web.DataReader('MS', 'yahoo', start, end)
gs = web.DataReader('GS', 'yahoo', start, end)
jpm = web.DataReader('JPM', 'yahoo', start, end)
c = web.DataReader('C', 'yahoo', start, end)

<h3>Compute the cross correlation coefficients on the stocks</h3>
<li>Create a df 'rets' with the one day percentage changes
<li>Calculate cross correlations on the rets df

In [3]:
rets = pd.DataFrame()
rets['ms'] = ms['Close'].pct_change()
rets['gs'] = gs['Close'].pct_change()
rets['jpm'] = jpm['Close'].pct_change()
rets['c'] = c['Close'].pct_change()

rets.corr()

,ms,gs,jpm,c
ms,1.000000,0.844340,0.801232,0.809949
gs,0.844340,1.000000,0.813289,0.792073
jpm,0.801232,0.813289,1.000000,0.849607
c,0.809949,0.792073,0.849607,1.000000


<h3>Select a pair of stocks to trade</h3>
<li>Choose the pair with the highest correlation
<li>You can just eyeball the result and pick the highest correlation pairs
<li>Create a new df 'pairs' with the returns columns of the highest correlation pairs

In [4]:
# pair with highest correlation = jpm, c

pairs = rets[['jpm', 'c']]
pairs

,jpm,c
Date,,
2013-01-02,NaN,NaN
2013-01-03,-0.002015,0.003394
2013-01-04,0.017725,0.025127
2013-01-07,0.001102,0.000943
2013-01-08,0.001982,-0.000236
...,...,...
2018-09-18,0.004041,0.006772
2018-09-19,0.029046,0.033072
2018-09-20,0.008587,0.014514


<h3>Calculate the mean and std dev of the columns (returns of the two stocks)</h3>
<li>Note that line 1 contains a NaN. You'll have to get rid of this line
<li>df.describe() returns summary stats for all cols in a dataframe. You can use this to extract means and standard deviations

In [5]:
pairs = pairs.iloc[1:]

c_mean = pairs.describe()['c']['mean']
c_std = pairs.describe()['c']['std']
j_mean = pairs.describe()['jpm']['mean']
j_std = pairs.describe()['jpm']['std']

<h3>Create a new column "Long_JPM_Short_C"</h3>
<li>If the return of C is greater than its mean + 0.25* std
<li> AND
<li>If the return of JPM is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [6]:
pairs['Long_JPM_Short_C'] = np.where(((pairs['c']>c_mean+c_std/4) & (pairs['jpm']<j_mean-j_std/4)), True, False)
pairs

<ipython-input-6-9389d861f547>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs['Long_JPM_Short_C'] = np.where(((pairs['c']>c_mean+c_std/4) & (pairs['jpm']<j_mean-j_std/4)), True, False)


,jpm,c,Long_JPM_Short_C
Date,,,
2013-01-03,-0.002015,0.003394,False
2013-01-04,0.017725,0.025127,False
2013-01-07,0.001102,0.000943,False
2013-01-08,0.001982,-0.000236,False
2013-01-09,-0.000659,-0.009892,False
...,...,...,...
2018-09-18,0.004041,0.006772,False
2018-09-19,0.029046,0.033072,False
2018-09-20,0.008587,0.014514,False


In [7]:
pairs.groupby('Long_JPM_Short_C').size()

Long_JPM_Short_C
False    1420
True       22
dtype: int64

<h3>Create a new column "Long_C_Short_JPM"</h3>
<li>If the return of JPM is greater than its mean + 0.25* std
<li> AND
<li>If the return of C is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [8]:
pairs['Long_C_Short_JPM'] = np.where(((pairs['jpm']>j_mean+j_std/4) & (pairs['c']<c_mean-c_std/4)), True, False)
pairs

<ipython-input-8-84f0cb89bb62>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs['Long_C_Short_JPM'] = np.where(((pairs['jpm']>j_mean+j_std/4) & (pairs['c']<c_mean-c_std/4)), True, False)


,jpm,c,Long_JPM_Short_C,Long_C_Short_JPM
Date,,,,
2013-01-03,-0.002015,0.003394,False,False
2013-01-04,0.017725,0.025127,False,False
2013-01-07,0.001102,0.000943,False,False
2013-01-08,0.001982,-0.000236,False,False
2013-01-09,-0.000659,-0.009892,False,False
...,...,...,...,...
2018-09-18,0.004041,0.006772,False,False
2018-09-19,0.029046,0.033072,False,False
2018-09-20,0.008587,0.014514,False,False


In [9]:
pairs.groupby('Long_C_Short_JPM').size()

Long_C_Short_JPM
False    1426
True       16
dtype: int64

<h3>Create a new column "Trade_return"</h3>
<li>If Long_JPM_Short_C is True and Long_C_Short_JPM is False, the value of this column should be the return on JPM minus the return on C
<li>If Long_JPM_Short_C is False and Long_C_Short_JPM is True, the value of this column should be the return on C minus the return on JPM
<li>Otherwise the value should be zero

In [11]:
# shift the 3,4th column down by one row, drop first row
pairs['Long_JPM_Short_C'] = pairs['Long_JPM_Short_C'].shift(1)
pairs['Long_C_Short_JPM'] = pairs['Long_C_Short_JPM'].shift(1)
pairs = pairs.iloc[1:]

<ipython-input-11-a3f4b7e3edf3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs['Long_JPM_Short_C'] = pairs['Long_JPM_Short_C'].shift(1)
<ipython-input-11-a3f4b7e3edf3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs['Long_C_Short_JPM'] = pairs['Long_C_Short_JPM'].shift(1)


In [12]:
# make function to calculate return
def get_return(x):
    if (x['Long_JPM_Short_C'] == True) and (x['Long_C_Short_JPM'] == False):
        return x['jpm'] - x['c']
    elif (x['Long_JPM_Short_C'] == False) and (x['Long_C_Short_JPM'] == True):
        return x['c'] - x['jpm']
    else:
        return 0

In [13]:
pairs['Trade_return'] = pairs.apply(get_return, axis = 1)
pairs

<ipython-input-13-39894f9fc2ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs['Trade_return'] = pairs.apply(get_return, axis = 1)


,jpm,c,Long_JPM_Short_C,Long_C_Short_JPM,Trade_return
Date,,,,,
2013-01-04,0.017725,0.025127,False,False,0.0
2013-01-07,0.001102,0.000943,False,False,0.0
2013-01-08,0.001982,-0.000236,False,False,0.0
2013-01-09,-0.000659,-0.009892,False,False,0.0
2013-01-10,0.014955,0.018792,False,False,0.0
...,...,...,...,...,...
2018-09-18,0.004041,0.006772,False,False,0.0
2018-09-19,0.029046,0.033072,False,False,0.0
2018-09-20,0.008587,0.014514,False,False,0.0


In [18]:
pairs[pairs['Trade_return']!=0]

,jpm,c,Long_JPM_Short_C,Long_C_Short_JPM,Trade_return
Date,,,,,
2013-01-24,0.003028,0.018563,True,False,-0.015534
2013-01-30,0.000212,-0.004981,False,True,-0.005193
2013-02-11,0.000617,0.011012,False,True,0.010395
2013-03-11,0.005578,0.019709,True,False,-0.014131
2013-04-02,0.009408,0.007308,False,True,-0.002101
2013-05-06,0.012823,0.010858,True,False,0.001965
2013-05-23,-0.005221,-0.009216,False,True,-0.003995
2013-07-02,0.005906,-0.002073,True,False,0.007978
2013-07-15,-0.001455,0.019681,True,False,-0.021137


<h3>Calculate the sum of this new column</h3>
<li>Are you going to get rich?

In [71]:
pairs['Trade_return'].sum()
# we get a negative value. 

-0.011651152935588605